# Module 04 — Neural Networks: Bike Sharing Demand Prediction

## Executive Summary — WelcomeBike DC Expansion

**Scenario**: WelcomeBike (Beijing-based bike-sharing company) expanding to Washington DC.
**Task**: Build a neural network to predict hourly bike rental volumes.
**Stakeholders**: Zhao (CEO), William (Investment Banker), Johnny (Data Science Intern)
**Framework**: scikit-learn MLPRegressor (neural network) + MinMaxScaler preprocessing
**Target**: Predict total hourly bike rentals (casual + registered)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, root_mean_squared_error, r2_score
from sklearn.neural_network import MLPRegressor
from sklearn.inspection import permutation_importance
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")
print("Using scikit-learn MLPRegressor for neural network")

## 1. Data Loading & Initial Exploration

In [ ]:
# Data URLs
BIKES_URL = "https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/bikes.csv"
HOLDOUT_URL = "https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/bikes_december.csv"
MINI_URL = "https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/biking_holdout_test_mini.csv"
MINI_ANSWERS_URL = "https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/biking_holdout_test_mini_answers.csv"

# Load training data
df = pd.read_csv(BIKES_URL)

print(f"Training data shape: {df.shape}")
print(f"Columns: {list(df.columns)}")
print(f"\nData types:\n{df.dtypes}")
df.head()

In [ ]:
# Basic statistics and missing values
print("=== Descriptive Statistics ===")
display(df.describe().round(2))
print(f"\n=== Missing Values ===")
print(df.isnull().sum())
print(f"\nTotal missing: {df.isnull().sum().sum()}")

## 2. Data Exploration

In [ ]:
# Create target variable
df['total'] = df['casual'] + df['registered']
print(f"Target (total hourly rentals) statistics:")
print(f"  Mean:   {df['total'].mean():.1f}")
print(f"  Median: {df['total'].median():.1f}")
print(f"  Std:    {df['total'].std():.1f}")
print(f"  Min:    {df['total'].min()}, Max: {df['total'].max()}")
print(f"  Skew:   {df['total'].skew():.2f}")

In [ ]:
# Rental patterns by hour, season, weather, and temperature
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Hourly pattern
hourly = df.groupby('hr')['total'].mean()
axes[0,0].bar(hourly.index, hourly.values, color='steelblue')
axes[0,0].set_xlabel('Hour of Day')
axes[0,0].set_ylabel('Average Total Rentals')
axes[0,0].set_title('Average Bike Rentals by Hour')

# Seasonal pattern
seasonal = df.groupby('season')['total'].mean()
season_labels = {1: 'Winter', 2: 'Spring', 3: 'Summer', 4: 'Fall'}
axes[0,1].bar([season_labels[s] for s in seasonal.index], seasonal.values, color='coral')
axes[0,1].set_ylabel('Average Total Rentals')
axes[0,1].set_title('Average Bike Rentals by Season')

# Weather pattern
weather = df.groupby('weathersit')['total'].mean()
weather_labels = {1: 'Clear', 2: 'Mist', 3: 'Light Rain', 4: 'Heavy Rain'}
w_labels = [weather_labels.get(w, str(w)) for w in weather.index]
axes[1,0].bar(w_labels, weather.values, color='mediumseagreen')
axes[1,0].set_ylabel('Average Total Rentals')
axes[1,0].set_title('Average Bike Rentals by Weather')

# Temperature vs rentals
axes[1,1].scatter(df['temp_c'], df['total'], alpha=0.05, s=5, color='steelblue')
axes[1,1].set_xlabel('Temperature (Celsius)')
axes[1,1].set_ylabel('Total Rentals')
axes[1,1].set_title('Temperature vs Bike Rentals')

plt.tight_layout()
plt.show()

In [ ]:
# Correlation heatmap
numeric_cols = ['hr', 'temp_c', 'feels_like_c', 'hum', 'windspeed', 'total']
corr = df[numeric_cols].corr()
plt.figure(figsize=(8, 6))
sns.heatmap(corr, annot=True, cmap='coolwarm', center=0, fmt='.2f')
plt.title('Feature Correlations')
plt.tight_layout()
plt.show()

## 3. Feature Engineering

Based on the data exploration, we engineer features to capture:
- **Temporal patterns**: Hour of day (cyclical + one-hot), day of week, month
- **Weather interactions**: Temperature x humidity, temperature x windspeed
- **Behavioral patterns**: Rush hour, nighttime, weekend indicators
- **Non-linear effects**: Temperature squared

The one-hot encoding of hour is critical because rental patterns are highly non-linear across hours (dual peaks at rush hours).

In [ ]:
def engineer_features(data, is_training=True):
    """Apply consistent feature engineering to training and holdout data."""
    d = data.copy()

    # Parse date and extract temporal features
    d['dteday'] = pd.to_datetime(d['dteday'], format='mixed')
    d['day_of_week'] = d['dteday'].dt.dayofweek  # 0=Monday, 6=Sunday
    d['month'] = d['dteday'].dt.month

    # Cyclical encoding for hour (captures 23->0 continuity)
    d['hr_sin'] = np.sin(2 * np.pi * d['hr'] / 24)
    d['hr_cos'] = np.cos(2 * np.pi * d['hr'] / 24)

    # Cyclical encoding for month
    d['month_sin'] = np.sin(2 * np.pi * d['month'] / 12)
    d['month_cos'] = np.cos(2 * np.pi * d['month'] / 12)

    # Behavioral indicators
    d['is_rush_hour'] = d['hr'].apply(lambda h: 1 if h in [7, 8, 9, 16, 17, 18, 19] else 0)
    d['is_night'] = d['hr'].apply(lambda h: 1 if h in [22, 23, 0, 1, 2, 3, 4, 5] else 0)
    d['is_weekend'] = (d['day_of_week'] >= 5).astype(int)

    # Interaction features
    d['temp_x_hum'] = d['temp_c'] * d['hum']
    d['temp_x_wind'] = d['temp_c'] * d['windspeed']
    d['temp_squared'] = d['temp_c'] ** 2

    # One-hot encode categoricals
    d = pd.get_dummies(d, columns=['season', 'weathersit', 'day_of_week', 'month'],
                       prefix=['season', 'weather', 'dow', 'month'], drop_first=False)

    # One-hot encode hour (captures non-linear hourly patterns)
    d['hr_int'] = d['hr'].astype(int)
    d = pd.get_dummies(d, columns=['hr_int'], prefix='hr', drop_first=False)

    # Drop columns not needed for modeling
    drop_cols = ['dteday', 'hr']
    if is_training:
        drop_cols += ['casual', 'registered']
    d = d.drop(columns=[c for c in drop_cols if c in d.columns], errors='ignore')
    return d

# Apply to training data
df_eng = engineer_features(df, is_training=True)
y = df_eng['total'].values
X = df_eng.drop(columns=['total'])

print(f"Engineered features: {X.shape[1]}")
print(f"Training samples: {X.shape[0]}")
print(f"\nFeature categories:")
print(f"  Continuous: temp_c, feels_like_c, hum, windspeed + cyclical + interactions")
print(f"  One-hot: season(4), weather(3-4), dow(7), month(12), hour(24)")
print(f"  Binary: holiday, workingday, is_rush_hour, is_night, is_weekend")

## 4. Preprocessing

Apply MinMaxScaler to continuous features as recommended by the course reading (Johnny's tip on feature scaling). Neural networks perform better when inputs are normalized to similar scales.

In [ ]:
# Identify continuous features for scaling
continuous_features = ['temp_c', 'feels_like_c', 'hum', 'windspeed',
                       'hr_sin', 'hr_cos', 'month_sin', 'month_cos',
                       'temp_x_hum', 'temp_x_wind', 'temp_squared']

# Fit scaler on training data
scaler = MinMaxScaler()
X[continuous_features] = scaler.fit_transform(X[continuous_features])

# Train/validation split (80/20)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training set:   {X_train.shape}")
print(f"Validation set: {X_val.shape}")
print(f"Target mean: {y_train.mean():.1f}, std: {y_train.std():.1f}")

## 5. Neural Network Model

We use scikit-learn's MLPRegressor — a multi-layer perceptron neural network for regression.
- **Architecture**: 3 hidden layers (128, 64, 32 neurons) with ReLU activation
- **Optimizer**: Adam with adaptive learning rate (initial lr=0.001)
- **Regularization**: L2 penalty (alpha=0.0001) + early stopping
- **Training**: Mini-batch (128 samples), up to 200 epochs with patience=15

In [ ]:
# Build the MLPRegressor neural network
model = MLPRegressor(
    hidden_layer_sizes=(128, 64, 32),
    activation='relu',
    solver='adam',
    learning_rate='adaptive',
    learning_rate_init=0.001,
    alpha=0.0001,  # L2 regularization
    max_iter=200,
    early_stopping=True,
    validation_fraction=0.1,
    n_iter_no_change=15,
    batch_size=128,
    random_state=42,
    verbose=True
)

print("Training neural network...")
model.fit(X_train, y_train)
print(f"\nConverged in {model.n_iter_} iterations")
print(f"Best validation score: {model.best_validation_score_:.4f}")

## 6. Training History

In [ ]:
# Plot training loss curve
plt.figure(figsize=(10, 5))
plt.plot(model.loss_curve_, label='Training Loss', color='steelblue')
plt.xlabel('Iteration')
plt.ylabel('MSE Loss')
plt.title('Training Loss Over Iterations')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Validation set performance
y_val_pred = np.maximum(model.predict(X_val), 0)  # No negative predictions

val_rmse = root_mean_squared_error(y_val, y_val_pred)
val_r2 = r2_score(y_val, y_val_pred)
val_mae = mean_absolute_error(y_val, y_val_pred)

print("=" * 50)
print("     VALIDATION SET PERFORMANCE")
print("=" * 50)
print(f"  RMSE:  {val_rmse:.2f}")
print(f"  R\u00b2:    {val_r2:.4f}")
print(f"  MAE:   {val_mae:.2f}")
print("=" * 50)

# Predicted vs Actual scatter
plt.figure(figsize=(8, 8))
plt.scatter(y_val, y_val_pred, alpha=0.1, s=5, color='steelblue')
max_val = max(y_val.max(), y_val_pred.max())
plt.plot([0, max_val], [0, max_val], 'r--', linewidth=2, label='Perfect prediction')
plt.xlabel('Actual Total Rentals')
plt.ylabel('Predicted Total Rentals')
plt.title(f'Predicted vs Actual \u2014 Validation Set\nRMSE={val_rmse:.2f}, R\u00b2={val_r2:.4f}')
plt.legend()
plt.tight_layout()
plt.show()

## 7. Mini Holdout Evaluation

Test our model against the mini holdout set (with known answers) to estimate grading performance.
The grading notebook evaluates: RMSE, R\u00b2, MAE, Median Absolute Error, and percent of predictions within 5%, 10%, 20% of actual.

In [ ]:
# Load mini holdout features and answers
mini_features = pd.read_csv(MINI_URL)
mini_answers = pd.read_csv(MINI_ANSWERS_URL)
mini_actual = (mini_answers['casual'] + mini_answers['registered']).values
print(f"Mini holdout samples: {len(mini_actual)}")

# Apply same feature engineering pipeline
mini_eng = engineer_features(mini_features, is_training=False)

# Ensure same columns as training (fill missing with 0)
for col in X.columns:
    if col not in mini_eng.columns:
        mini_eng[col] = 0
mini_eng = mini_eng[X.columns]

# Scale with the SAME fitted scaler
mini_eng[continuous_features] = scaler.transform(mini_eng[continuous_features])

# Predict
mini_pred = np.maximum(model.predict(mini_eng), 0)

# ===== KEY METRICS =====
mini_rmse = root_mean_squared_error(mini_actual, mini_pred)
mini_r2 = r2_score(mini_actual, mini_pred)
mini_mae = mean_absolute_error(mini_actual, mini_pred)

print()
print("=" * 55)
print("     MINI HOLDOUT PERFORMANCE (vs known answers)")
print("=" * 55)
print(f"  RMSE:              {mini_rmse:.2f}")
print(f"  R\u00b2:                {mini_r2:.4f}")
print(f"  MAE:               {mini_mae:.2f}")
print("-" * 55)

for pct in [5, 10, 20]:
    within = np.mean(np.abs(mini_pred - mini_actual) <= mini_actual * (pct/100)) * 100
    print(f"  Within {pct}%:         {within:.1f}%")

print("=" * 55)

# Save mini holdout predictions
mini_pred_int = np.round(mini_pred).astype(int)
mini_predictions_df = pd.DataFrame({'predictions': mini_pred_int})
mini_predictions_df.to_csv('team8-module4-mini-predictions.csv', index=False)
print(f"\nSaved mini holdout predictions to team8-module4-mini-predictions.csv")
print(f"Shape: {mini_predictions_df.shape}")

In [ ]:
# Mini holdout visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Scatter: predicted vs actual with 20% threshold coloring
pct_diff = np.abs(mini_pred - mini_actual) / np.maximum(mini_actual, 1)
colors = ['steelblue' if p <= 0.2 else 'coral' for p in pct_diff]
axes[0].scatter(mini_actual, mini_pred, c=colors, alpha=0.5, s=20)
max_val = max(mini_actual.max(), mini_pred.max())
axes[0].plot([0, max_val], [0, max_val], 'r--', linewidth=2)
axes[0].set_xlabel('Actual Total Rentals')
axes[0].set_ylabel('Predicted Total Rentals')
axes[0].set_title(f'Mini Holdout: Predicted vs Actual\nRMSE={mini_rmse:.2f}, R\u00b2={mini_r2:.4f}')

# Time series: actual vs predicted
axes[1].plot(mini_actual, label='Actual', alpha=0.8, color='steelblue')
axes[1].plot(mini_pred, label='Predicted', alpha=0.8, color='coral')
axes[1].set_xlabel('Sample Index (hourly)')
axes[1].set_ylabel('Total Rentals')
axes[1].set_title('Mini Holdout: Actual vs Predicted Over Time')
axes[1].legend()

plt.tight_layout()
plt.show()

## 8. Full Holdout Predictions (November/December)

In [ ]:
# Load full holdout dataset
holdout = pd.read_csv(HOLDOUT_URL)
print(f"Holdout data shape: {holdout.shape}")

# Apply same feature engineering pipeline
holdout_eng = engineer_features(holdout, is_training=False)
for col in X.columns:
    if col not in holdout_eng.columns:
        holdout_eng[col] = 0
holdout_eng = holdout_eng[X.columns]
holdout_eng[continuous_features] = scaler.transform(holdout_eng[continuous_features])

# Predict
holdout_pred = np.maximum(model.predict(holdout_eng), 0)
holdout_pred = np.round(holdout_pred).astype(int)

print(f"Predictions generated: {len(holdout_pred)}")
print(f"Prediction range: {holdout_pred.min()} - {holdout_pred.max()}")
print(f"Mean prediction: {holdout_pred.mean():.1f}")

In [ ]:
# Save predictions CSV for submission
predictions_df = pd.DataFrame({'predictions': holdout_pred})
predictions_df.to_csv('team8-module4-predictions.csv', index=False)
print(f"Saved predictions to team8-module4-predictions.csv")
print(f"Shape: {predictions_df.shape}")
print(f"\nFirst 10 predictions:")
display(predictions_df.head(10))

## 9. Model Analysis & Visualizations

In [ ]:
# Residual analysis on validation set
residuals = y_val - y_val_pred

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Residual distribution
axes[0].hist(residuals, bins=50, color='steelblue', edgecolor='black', alpha=0.7)
axes[0].axvline(x=0, color='red', linestyle='--')
axes[0].set_xlabel('Residual (Actual - Predicted)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Residual Distribution')

# Residuals vs Predicted
axes[1].scatter(y_val_pred, residuals, alpha=0.05, s=5, color='steelblue')
axes[1].axhline(y=0, color='red', linestyle='--')
axes[1].set_xlabel('Predicted Values')
axes[1].set_ylabel('Residuals')
axes[1].set_title('Residuals vs Predicted Values')

# Hourly prediction patterns
y_all_pred = np.maximum(model.predict(X), 0)
pred_df = pd.DataFrame({'actual': y, 'predicted': y_all_pred, 'hr': df['hr'].values})
hourly_comp = pred_df.groupby('hr').agg({'actual': 'mean', 'predicted': 'mean'})
axes[2].bar(hourly_comp.index - 0.2, hourly_comp['actual'], 0.4, label='Actual', color='steelblue')
axes[2].bar(hourly_comp.index + 0.2, hourly_comp['predicted'], 0.4, label='Predicted', color='coral')
axes[2].set_xlabel('Hour of Day')
axes[2].set_ylabel('Average Total Rentals')
axes[2].set_title('Actual vs Predicted by Hour')
axes[2].legend()

plt.tight_layout()
plt.show()

In [ ]:
# Permutation importance
sample_idx = np.random.choice(len(X_val), min(3000, len(X_val)), replace=False)
X_sample = X_val.iloc[sample_idx]
y_sample = y_val[sample_idx]

perm_imp = permutation_importance(model, X_sample, y_sample, n_repeats=5,
                                   random_state=42, scoring='neg_mean_absolute_error')

# Top 15 features
imp_df = pd.DataFrame({
    'feature': X.columns,
    'importance': perm_imp.importances_mean
}).sort_values('importance', ascending=True).tail(15)

plt.figure(figsize=(10, 8))
plt.barh(imp_df['feature'], imp_df['importance'], color='steelblue')
plt.xlabel('Importance (decrease in MAE when permuted)')
plt.title('Top 15 Feature Importances (Permutation)')
plt.tight_layout()
plt.show()

## 10. Discussion Questions

### Q1: Network Architecture (Zhao, CEO)
**"How many layers should the network have, and which hyperparameter has the most potential for improvement?"**

Our final model uses **3 hidden layers (128 \u2192 64 \u2192 32 neurons)** with ReLU activation. We found that **(B) learning rate and optimizer selection** had the most impact on model performance. The Adam optimizer with an initial learning rate of 0.001 and adaptive scheduling gave the best results. We also found that adding one-hot encoding for the hour feature (24 binary columns) was more impactful than adding more layers beyond 3.

### Q2: Feature Engineering (Johnny, Intern)
**"How should we handle the temperature features?"**

We kept both `temp_c` (actual temperature) and `feels_like_c` (feels-like temperature) and applied MinMaxScaler normalization as recommended by the course reading. We also created interaction features (`temp_c \u00d7 humidity`, `temp_c \u00d7 windspeed`) and a quadratic term (`temp_squared`) to capture the non-linear relationship between temperature and demand (rentals peak at moderate temperatures, not at extremes).

### Q3: Learning Rate Optimization (Zhao, CEO)
**"What approach will you take to find the optimal learning rate?"**

We used scikit-learn's adaptive learning rate approach:
1. Start with Adam optimizer's default learning rate (0.001)
2. The `adaptive` learning rate policy automatically reduces the rate when training loss plateaus
3. Early stopping with patience of 15 epochs prevents overfitting
4. This avoids exhaustive grid search while finding good convergence

### Q4: Loss Function (Johnny, Intern)
**"What loss function will you use? How will we know if the model has strong predictive power?"**

We use **Mean Squared Error (MSE)** as the loss function because it penalizes large prediction errors more heavily, which is important for demand forecasting where big misses are costly.

We evaluate with multiple metrics:
- **RMSE** (root of MSE): interpretable in same units as target
- **R-squared**: proportion of variance explained (1.0 = perfect)
- **MAE**: average absolute error in bike count

### Q5: Ethics \u2014 Insurance Pricing (William, Investment Banker)
**"Should we predict damage likelihood based on user profile data for insurance pricing?"**

We recommend **(D) Use real-time GPS behavioral monitoring** instead of profile-based data. Using name, birthday, sex, and address for damage prediction risks:
- **Discriminatory pricing** based on demographics (similar to redlining)
- **Privacy violations** even with consent (consent can be coerced)
- **Legal liability** under fair lending and anti-discrimination laws

Instead, GPS-based behavioral monitoring (with transparent user consent) assesses actual riding behavior, not demographic proxies.

### Q6: Post-Pandemic Recovery (Zhao, CEO)
**"When should bikes be pulled for cleaning? Are there lasting pandemic effects?"**

Based on weather and usage patterns in the data:
- **Cleaning schedule**: Rotate bikes during **low-demand hours** (midnight\u20135 AM) and during **severe weather events** (weathersit = 3 or 4) when demand drops significantly
- **Pandemic effects**: Casual ridership has recovered, but commuter patterns (registered users) may have permanently shifted due to remote work. Our model shows that `is_weekend` and `is_rush_hour` are important features, and changes in commuter behavior would affect rush hour demand
- **Recommendation**: Monitor the ratio of casual-to-registered riders over time to detect structural changes

## 11. Summary & Results

### Model Performance

| Metric | Validation | Mini Holdout |
|--------|-----------|--------------|
| **RMSE** | ~152 | **~224** |
| **R-squared** | ~0.804 | **~0.622** |
| **MAE** | ~95 | **~164** |

### Key Model Features
- 3-layer MLP neural network (128, 64, 32) with ReLU activation
- 67 engineered features including one-hot hour, cyclical encodings, and interaction terms
- MinMaxScaler preprocessing on continuous features
- Adam optimizer with adaptive learning rate and early stopping

### Business Recommendations for WelcomeBike DC
1. **Staffing**: Deploy extra bikes during rush hours (7-9 AM, 4-7 PM) on workdays
2. **Weather response**: Reduce inventory during rain/snow events (weathersit 3-4)
3. **Seasonal planning**: Summer and fall have highest demand; winter requires reduced fleet
4. **Location strategy**: Focus stations near transit hubs and popular commute routes
5. **Revenue optimization**: Dynamic pricing during peak demand hours could increase revenue